In [1]:
import tensorflow as tf

import os
import tensorflow_datasets as tfds

In [2]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

All devices:  [LogicalDevice(name='/device:TPU:0', device_type='TPU'), LogicalDevice(name='/device:TPU:1', device_type='TPU'), LogicalDevice(name='/device:TPU:2', device_type='TPU'), LogicalDevice(name='/device:TPU:3', device_type='TPU'), LogicalDevice(name='/device:TPU:4', device_type='TPU'), LogicalDevice(name='/device:TPU:5', device_type='TPU'), LogicalDevice(name='/device:TPU:6', device_type='TPU'), LogicalDevice(name='/device:TPU:7', device_type='TPU')]


In [3]:
strategy = tf.distribute.TPUStrategy(resolver)

In [4]:
!pip install hickle
import hickle as hkl
import os
import keras
from keras import backend as K
import matplotlib.pyplot as plt
from keras import layers,Input,Sequential
from keras.layers import Dense,Flatten,Reshape,Conv2DTranspose,Conv2D,MaxPooling2D,Reshape,Resizing,Dropout,UpSampling2D,BatchNormalization
from tensorflow.keras import datasets, layers, models, losses, Model
from numpy import random
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential,Model
from keras.layers import Multiply, Add
import matplotlib.pyplot as plt
from google.colab  import drive
drive.mount("/gdrive")
%cd /gdrive

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 1.4 MB/s eta 0:00:00
Mounted at /gdrive
/gdrive


In [5]:
load_hkl = hkl.load("/gdrive/My Drive/Yedek/TF_Datasets_v3/Frontal_vae.hkl")
X_train=load_hkl['xtrain']
X_train = X_train.astype('float32') / 255.0

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/gdrive/My Drive/Yedek/TF_Datasets_v3/Frontal_vae.hkl', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
with strategy.scope():
  latent_dim = 2
  encoder_inputs= keras.Input(shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3]))

  x = layers.Conv2D(16, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
  x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(x)
  x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
  x = layers.Flatten()(x)
  x = layers.Dense(64, activation="relu")(x)
  z_mean = layers.Dense(latent_dim, name="z_mean")(x)
  z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)



In [ ]:
class Sampler(layers.Layer):
  def call(self, z_mean, z_log_var):
      batch_size = tf.shape(z_mean)[0]
      z_size = tf.shape(z_mean)[1]
      epsilon = tf.random.normal(shape=(batch_size, z_size))
      return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [ ]:
with strategy.scope():
  latent_inputs = keras.Input(shape=(latent_dim,))

  x = layers.Dense(60 * 80* 64, activation="relu")(latent_inputs)
  x = layers.Reshape((60, 80, 64))(x)
  x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
  x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
  x = layers.Conv2DTranspose(16, 3, activation="relu", strides=2, padding="same")(x)
  decoder_outputs = layers.Conv2D(3, 3, activation="sigmoid", padding="same")(x)


In [ ]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
      super().__init__(**kwargs)
      self.encoder = encoder
      self.decoder = decoder
      self.sampler = Sampler()
      self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
      self.reconstruction_loss_tracker = keras.metrics.Mean(name="reconstruction_loss")
      self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [self.total_loss_tracker,self.reconstruction_loss_tracker,self.kl_loss_tracker]

    def train_step(self, data):
      with tf.GradientTape() as tape:
        z_mean, z_log_var = self.encoder(data)
        z = self.sampler(z_mean, z_log_var)
        reconstruction = decoder(z)
        reconstruction_loss = tf.reduce_mean(
            tf.reduce_sum(
                keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.SUM)(data, reconstruction)
            )
        )
        kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean)-tf.exp(z_log_var))
        total_loss = reconstruction_loss + kl_loss

        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)

        return {
        "total_loss": self.total_loss_tracker.result(),
        "reconstruction_loss": self.reconstruction_loss_tracker.result(),
        "kl_loss": self.kl_loss_tracker.result(),}

In [ ]:
with strategy.scope():
  decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
  encoder = keras.Model(encoder_inputs, [z_mean, z_log_var], name="encoder")
  vae = VAE(encoder, decoder)
  vae.compile(optimizer=keras.optimizers.Adam(), run_eagerly=False)
vae.fit(X_train,  epochs=30, batch_size=4)

In [ ]:
import matplotlib.pyplot as plt
n = 30
digit_size = 28
figure = np.zeros((digit_size * n, digit_size * n, 3))
grid_x = np.linspace(-1, 1, n)
grid_y = np.linspace(-1, 1, n)[::-1]
for i, yi in enumerate(grid_y):
 for j, xi in enumerate(grid_x):
    z_sample = np.array([[xi, yi]])
    x_decoded = vae.decoder.predict(z_sample)
    digit = x_decoded[0].reshape(digit_size, digit_size)
    figure[
    i * digit_size : (i + 1) * digit_size,